# Data processing: Pandas 

### Acknowledgments & Credits

This lesson is adapted largely from the excellent curriculum materials by Cliburn Chan (2021) at https://github.com/cliburn/bios-823-2021/ under the MIT License.

**References**

- [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)
- [Python for Data Analysis, 2nd Edition](https://github.com/wesm/pydata-book)

In [ ]:
import numpy as np
import pandas as pd

## Series and Data Frames

### Series objects

A `Series` is like a vector. All elements must have the same type or are nulls.

In [ ]:
s = pd.Series([1,1,2,3] + [None])
s

### Size

In [ ]:
s.size

### Unique Values and Counts

In [ ]:
s.value_counts()

In [ ]:
s.nunique(), s.unique()

### Special types of series

#### Strings

In [ ]:
words = 'the quick brown fox jumps over the lazy dog'.split()
s1 = pd.Series([' '.join(item) for item in zip(words[:-1], words[1:])])
s1

In [ ]:
s1.str.upper()

In [ ]:
s1.str.split()

In [ ]:
s1.str.split().str[1]

### Categories

In [ ]:
b = pd.Series(['Adenine', 'Cytosine', 'Guanine', 'Thymine', 'Uracil'])
b

In [ ]:
b = b.astype('category')
b

In [ ]:
b.cat.categories

In [ ]:
b.cat.codes

In [ ]:
b = pd.Series(list('ACGTU')).astype('category')
b

In [ ]:
b.cat.categories

In [ ]:
b = pd.Series(list(b'ACGTU')).astype('category')
b

### DataFrame objects

A `DataFrame` is like a matrix. Columns in a `DataFrame` are `Series`.

- Each row in a DataFrame represents an **observation**
- Each column in a DataFrame represents a **feature** (or **variable**)
- Each cell in a DataFrame represents a **value**

In [ ]:
df = pd.DataFrame(dict(num=[1,2,3] + [None]))
df

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
def sklearnds2df(ds):
    df = pd.DataFrame(data=ds['data'], columns=ds['feature_names'])
    df['target'] = pd.Series(pd.Categorical.from_codes(ds['target'],
                                                       categories=iris['target_names']))
    return df

In [ ]:
df = ds2df(iris)
df

### Index

Row and column identifiers are of `Index` type.

Somewhat confusingly, index is also a a synonym for the row identifiers.

In [ ]:
df.index

#### Making an index into a column

In [ ]:
df.reset_index(drop=True)
df

### Columns

This is just a different index object

In [ ]:
df.columns

### Getting raw values

Sometimes you just want a `numpy` array, and not a `pandas` object.

In [ ]:
df.values

## Indexing Data Frames

### Implicit defaults

if you provide a slice, it is assumed that you are asking for rows.

In [ ]:
df[1:3]

If you provide a singe value or list, it is assumed that you are asking for columns.

In [ ]:
df[['sepal length (cm)', 'sepal width (cm)']]

### Indexing by location

This is similar to `numpy` indexing

In [ ]:
df.iloc[1:3, :]

In [ ]:
df.iloc[1:3, 1:4:2]

### Indexing by name

In [ ]:
df.loc[1:3, 'sepal length (cm)':'petal length (cm)']

**Warning**: When using `loc`, the row slice indicates row names, not positions.

In [ ]:
df1 = df.copy()
df1.index = df.index + 1
df1

In [ ]:
df1.loc[1:3, 'sepal length (cm)':'petal length (cm)']

In [ ]:
df1.iloc[1:3, 0:3]

## Structure of a Data Frame

### Data types

In [ ]:
df.dtypes

### Converting data types

#### Using `astype`, including on multiple columns

In [ ]:
df1 = df.astype({'sepal length (cm)': int, 'sepal width (cm)':int})
df1

In [ ]:
df1.dtypes

### Basic properties

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.sample(n=3)

In [ ]:
df.sample(frac=0.1)

## Selecting, Renaming and Removing Columns

### Selecting columns

In [ ]:
df.filter(regex='\slength\s')

### By data type

In [ ]:
df.select_dtypes(include=np.number)

In [ ]:
df.select_dtypes(exclude=['category'])

#### Note that you can also use regular string methods on the columns

In [ ]:
df.loc[:, df.columns.str.contains('length')]

### Renaming columns

In [ ]:
df.rename(dict(target='species'), axis=1)

In [ ]:
df.columns = df.columns.str.replace(" (cm)", "").str.replace(" ", "_")
df

#### You can also use regular indexing

In [ ]:
df.loc[:, ~df.columns.str.contains('sepal')]

## Selecting, Renaming and Removing Rows

### Selecting rows

In [ ]:
df[df.sepal_length.between(5,6)]

In [ ]:
df.query('5 <= sepal_length <= 6')

## Sorting Data Frames

### Sort on indexes

In [ ]:
df.sort_index(axis=1)

In [ ]:
df.sort_index(axis=0, ascending=False)

### Sort on values

In [ ]:
df.sort_values(by=['sepal_length', 'petal_length'], ascending=[True, False])

## Summarizing

### Apply an aggregation function

In [ ]:
df.agg(['target', np.sum, np.mean])

In [ ]:
df.agg({'weight': ['median', 'sum'], 'height': ['min', 'max']})

## Split-Apply-Combine

We often want to perform subgroup analysis (conditioning by some discrete or categorical variable). This is done with `groupby` followed by an aggregate function. Conceptually, we split the data frame into separate groups, apply the aggregate function to each group separately, then combine the aggregated results back into a single data frame.

In [ ]:
df['treatment'] = list('ababa')

In [ ]:
df

In [ ]:
grouped = df.groupby('treatment')

In [ ]:
grouped.get_group('a')

In [ ]:
grouped.mean()

### Using `agg` with `groupby`

In [ ]:
grouped.agg('mean')

In [ ]:
grouped.agg(['mean', 'std'])

In [ ]:
grouped.agg({'weight': ['mean', 'std'], 
             'height': ['min', 'max'], 'bmi': lambda x: (x**2).sum()})

### Using `transform` wtih `groupby`

When you apply a transform with a grouped object, it returns the same value for each member in a group - all rows are represented.

In [ ]:
g_mean = grouped[['weight', 'height']].transform(np.mean)
g_mean

In [ ]:
g_std = grouped[['weight', 'height']].transform(np.std)
g_std

In [ ]:
(df[['weight', 'height']] - g_mean)/g_std

## Combining Data Frames

In [ ]:
df

In [ ]:
df1 =  df.iloc[3:].copy()

In [ ]:
df1.drop('something', axis=1, inplace=True)
df1

### Adding rows

Note that `pandas` aligns by column indexes automatically.

In [ ]:
df.append(df1, sort=False)

In [ ]:
pd.concat([df, df1], sort=False)

### Adding columns

In [ ]:
df.pid

In [ ]:
df2 = pd.DataFrame(OrderedDict(pid=[649, 533, 400, 600], age=[23,34,45,56]))

In [ ]:
df2.pid

In [ ]:
df.pid = df.pid.astype('int')

In [ ]:
pd.merge(df, df2, on='pid', how='inner')

In [ ]:
pd.merge(df, df2, on='pid', how='left')

In [ ]:
pd.merge(df, df2, on='pid', how='right')

In [ ]:
pd.merge(df, df2, on='pid', how='outer')

### Merging on the index

In [ ]:
df1 = pd.DataFrame(dict(x=[1,2,3]), index=list('abc'))
df2 = pd.DataFrame(dict(y=[4,5,6]), index=list('abc'))
df3 = pd.DataFrame(dict(z=[7,8,9]), index=list('abc'))

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

In [ ]:
df1.join([df2, df3])

## Reshaping Data Frames

Sometimes we need to make rows into columns or vice versa.

### Converting multiple columns into a single column

This is often useful if you need to condition on some variable.

In [ ]:
url = 'https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv'
iris = pd.read_csv(url)

In [ ]:
iris.head()

In [ ]:
iris.shape

In [ ]:
df_iris = pd.melt(iris, id_vars='species')

In [ ]:
df_iris.sample(10)

## Pivoting

Sometimes we need to convert categorical values in a column into separate columns. This is often done at the same time as performing a summary.

In [ ]:
df_iris.pivot_table(index='variable', columns='species', values='value', aggfunc='mean')

## Functional style - `apply`, `map`

`apply` can be used to apply a custom function

In [ ]:
scores = pd.DataFrame(
    np.around(np.clip(np.random.normal(90, 10, (5,3)), 0, 100), 1),
    columns = ['math', 'stat', 'biol'],
    index = ['anne', 'bob', 'charles', 'dirk', 'edgar']
)
scores

In [ ]:
def convert_grade(score):
    return np.choose(
        pd.cut(score, [-1, 60, 70, 80, 90, 100], labels=False),
        ['F', 'D', 'C', 'B', 'A']
    )

In [ ]:
scores.apply(convert_grade_2)

`apply` can be used to avoid explicit looping. It is also very handy for reductions along margins.

In [ ]:
scores.apply(np.mean, axis=0)

If all else fails, you can loop over `pandas` data frames. Loops are frowned upon because they are not efficient, but sometimes pragmatism beats elegance.